# Avaliação Prática – Análise Econômica e Criptofinanceira com Alpha Vantage (Inflação, Câmbio e Bitcoin)

## Pré-Processamento de Dados

### Importação de Bibliotecas

In [1]:
!pip install plotly --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import pickle

### Consumo da API Alpha Vantage para Carregamento das Bases de Dados e Conversão para Pandas Dataframe

#### Função "INFLATION" da API para Obteção de Dados sobre a Inflação

In [3]:
url_inflation = 'https://www.alphavantage.co/query?function=INFLATION&apikey=NES1FN2QJITLR295'

In [ ]:
r = requests.get(url_inflation)

In [4]:
data = r.json()

{'name': 'Inflation - US Consumer Prices',
 'interval': 'annual',
 'unit': 'percent',
 'data': [{'date': '2024-01-01', 'value': '2.94952520485207'},
  {'date': '2023-01-01', 'value': '4.11633838374488'},
  {'date': '2022-01-01', 'value': '8.00279982052121'},
  {'date': '2021-01-01', 'value': '4.69785886363742'},
  {'date': '2020-01-01', 'value': '1.23358439630629'},
  {'date': '2019-01-01', 'value': '1.81221007526021'},
  {'date': '2018-01-01', 'value': '2.44258329692817'},
  {'date': '2017-01-01', 'value': '2.13011000365961'},
  {'date': '2016-01-01', 'value': '1.26158320570536'},
  {'date': '2015-01-01', 'value': '0.118627135552451'},
  {'date': '2014-01-01', 'value': '1.62222297740817'},
  {'date': '2013-01-01', 'value': '1.46483265562717'},
  {'date': '2012-01-01', 'value': '2.06933726526067'},
  {'date': '2011-01-01', 'value': '3.156841568622'},
  {'date': '2010-01-01', 'value': '1.6400434423899'},
  {'date': '2009-01-01', 'value': '-0.355546266299747'},
  {'date': '2008-01-01', '

In [5]:
inflation_data = data['data']
inflation_df = pd.DataFrame(inflation_data)
inflation_df

,date,value
0,2024-01-01,2.94952520485207
1,2023-01-01,4.11633838374488
2,2022-01-01,8.00279982052121
3,2021-01-01,4.69785886363742
4,2020-01-01,1.23358439630629
...,...,...
60,1964-01-01,1.27891156462591
61,1963-01-01,1.23966942148753
62,1962-01-01,1.19877334820186
63,1961-01-01,1.07072414764724


In [6]:
inflation_df = inflation_df.set_index('date')
inflation_df.index.name = None
inflation_df

,value
2024-01-01,2.94952520485207
2023-01-01,4.11633838374488
2022-01-01,8.00279982052121
2021-01-01,4.69785886363742
2020-01-01,1.23358439630629
...,...
1964-01-01,1.27891156462591
1963-01-01,1.23966942148753
1962-01-01,1.19877334820186
1961-01-01,1.07072414764724


#### Função "FX_DAILY" da API para Obtenção de Dados sobre Câmbio

In [7]:
url_fx_currency = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=BRL&apikey=NES1FN2QJITLR295'

In [ ]:
r = requests.get(url_fx_currency)

In [11]:
data = r.json()

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'USD',
  '3. To Symbol': 'BRL',
  '4. Output Size': 'Compact',
  '5. Last Refreshed': '2025-04-24',
  '6. Time Zone': 'UTC'},
 'Time Series FX (Daily)': {'2025-04-24': {'1. open': '5.71110',
   '2. high': '5.71160',
   '3. low': '5.66090',
   '4. close': '5.68250'},
  '2025-04-23': {'1. open': '5.72010',
   '2. high': '5.72750',
   '3. low': '5.65680',
   '4. close': '5.71110'},
  '2025-04-22': {'1. open': '5.78070',
   '2. high': '5.80670',
   '3. low': '5.71750',
   '4. close': '5.72100'},
  '2025-04-21': {'1. open': '5.80100',
   '2. high': '5.80700',
   '3. low': '5.77830',
   '4. close': '5.78070'},
  '2025-04-18': {'1. open': '5.80100',
   '2. high': '5.81140',
   '3. low': '5.78760',
   '4. close': '5.80670'},
  '2025-04-17': {'1. open': '5.85620',
   '2. high': '5.88790',
   '3. low': '5.79580',
   '4. close': '5.80100'},
  '2025-04-16': {'1. open': '5.89290',
   '2. high': '5.91

In [13]:
time_series = data['Time Series FX (Daily)']
fx_df = pd.DataFrame.from_dict(time_series, orient='index')
fx_df.index = pd.to_datetime(fx_df.index)
fx_df = fx_df.sort_index()
fx_df

,1. open,2. high,3. low,4. close
2024-12-06,5.99440,6.09470,5.98280,6.08940
2024-12-09,6.08940,6.08980,6.03650,6.07890
2024-12-10,6.07860,6.08290,6.01730,6.05990
2024-12-11,6.05990,6.07280,5.94000,5.95410
2024-12-12,5.95430,6.04760,5.86590,5.99230
...,...,...,...,...
2025-04-18,5.80100,5.81140,5.78760,5.80670
2025-04-21,5.80100,5.80700,5.77830,5.78070
2025-04-22,5.78070,5.80670,5.71750,5.72100
2025-04-23,5.72010,5.72750,5.65680,5.71110


#### Função "DIGITAL_CURRENCY_DAILY" da API para Obtenção de Dados sobre Bitcoin

In [14]:
url_btc = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey=NES1FN2QJITLR295'

In [15]:
r = requests.get(url_btc)

In [21]:
data = r.json()

In [17]:
btc_time_series = data['Time Series (Digital Currency Daily)']
btc_df = pd.DataFrame.from_dict(btc_time_series, orient='index')
btc_df.index = pd.to_datetime(btc_df.index)
btc_df = btc_df.sort_index()
btc_df

,1. open,2. high,3. low,4. close,5. volume
2024-05-10,58522.09000000,58882.51000000,55876.05000000,56514.18000000,447.06249428
2024-05-11,56524.04000000,57167.19000000,56208.31000000,56496.94000000,198.07642651
2024-05-12,56516.31000000,57435.99000000,56299.00000000,57075.98000000,123.98655238
2024-05-13,57085.97000000,58770.58000000,56418.03000000,58293.80000000,325.85438232
2024-05-14,58298.94000000,58478.73000000,56500.00000000,56924.05000000,320.35856449
...,...,...,...,...,...
2025-04-20,74671.04000000,74889.29000000,73717.90000000,74465.95000000,103.73595565
2025-04-21,74472.62000000,76933.38000000,74357.03000000,75995.58000000,369.45616882
2025-04-22,75997.36000000,82673.74000000,75827.31000000,82333.82000000,550.73983560
2025-04-23,82339.52000000,83264.20000000,80800.00000000,82718.26000000,490.53056792


#### Visualização dos Dataframes

In [18]:
inflation_df

,value
2024-01-01,2.94952520485207
2023-01-01,4.11633838374488
2022-01-01,8.00279982052121
2021-01-01,4.69785886363742
2020-01-01,1.23358439630629
...,...
1964-01-01,1.27891156462591
1963-01-01,1.23966942148753
1962-01-01,1.19877334820186
1961-01-01,1.07072414764724


In [19]:
fx_df

,1. open,2. high,3. low,4. close
2024-12-06,5.99440,6.09470,5.98280,6.08940
2024-12-09,6.08940,6.08980,6.03650,6.07890
2024-12-10,6.07860,6.08290,6.01730,6.05990
2024-12-11,6.05990,6.07280,5.94000,5.95410
2024-12-12,5.95430,6.04760,5.86590,5.99230
...,...,...,...,...
2025-04-18,5.80100,5.81140,5.78760,5.80670
2025-04-21,5.80100,5.80700,5.77830,5.78070
2025-04-22,5.78070,5.80670,5.71750,5.72100
2025-04-23,5.72010,5.72750,5.65680,5.71110


In [20]:
btc_df

,1. open,2. high,3. low,4. close,5. volume
2024-05-10,58522.09000000,58882.51000000,55876.05000000,56514.18000000,447.06249428
2024-05-11,56524.04000000,57167.19000000,56208.31000000,56496.94000000,198.07642651
2024-05-12,56516.31000000,57435.99000000,56299.00000000,57075.98000000,123.98655238
2024-05-13,57085.97000000,58770.58000000,56418.03000000,58293.80000000,325.85438232
2024-05-14,58298.94000000,58478.73000000,56500.00000000,56924.05000000,320.35856449
...,...,...,...,...,...
2025-04-20,74671.04000000,74889.29000000,73717.90000000,74465.95000000,103.73595565
2025-04-21,74472.62000000,76933.38000000,74357.03000000,75995.58000000,369.45616882
2025-04-22,75997.36000000,82673.74000000,75827.31000000,82333.82000000,550.73983560
2025-04-23,82339.52000000,83264.20000000,80800.00000000,82718.26000000,490.53056792
